<a href="https://colab.research.google.com/github/LarsBryld/socialdataanalysis2020/blob/master/Oxford.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

ox = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv" , parse_dates=["Date"])

covid = pd.read_csv("https://open-covid-19.github.io/data/data.csv", parse_dates=["Date"])



#Define focus contries based on stock indecies
focuscountries = set(['China', 'Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Italy', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Netherlands', 'Poland', 'Portugal', 'Spain',
                  'Sweden', 'Switzerland', 'United Kingdom', 'Canada', 'United States', 'Australia', 'New Zealand', 'Hong Kong', 'Japan', 'South Korea', 'Singapore',
                  'Taiwan', 'Brazil', 'Israel', 'India', 'Mexico', 'South Africa', 'Russia'])


#Create dataframes of only focus countries
#ox = ox[ox['CountryName'].isin(focuscountries)]
covid = covid[covid['CountryName'].isin(focuscountries)]


#Sometimes, due to data entry errors, we have isgeneral=1 when the underlying indicator is 0 or null. These cases should be disregarded. Only use isgeneral=1 values when the underlying indicator is 1 or higher.
cond1 = ((ox['CountryName'] == 'Denmark'))

#test = ox.where(cond=cond1,inplace=False)


ox = ox[(ox['S1_IsGeneral'] != 1) & (ox['S1_School closing'] > 0)]
#ox =  (ox['S2_IsGeneral'] != 1) & (ox['S2_Workplace closing'] > 0) 
#ox = ox[(ox['S2_IsGeneral'] != 1) & (ox['S2_Workplace closing'] > 0)]
#ox = ox[(ox['S3_IsGeneral'] != 1) & (ox['S3_Cancel public events'] > 0)]
#ox = ox[(ox['S4_IsGeneral'] != 1) & (ox['S4_Close public transport'] > 0)]
#ox = ox[(ox['S5_IsGeneral'] != 1) & (ox['S5_Public information campaigns'] > 0)]
#ox = ox[(ox['S6_IsGeneral'] != 1) & (ox['S6_Restrictions on internal movement'] > 0)]
#drop2 = ox[(ox['S1_IsGeneral'] == 1) & (ox['S1_School closing'] == 0)] 



#Nine of the indicators (S1–S7, S12 and S13) take policies such as school closures, travel bans, etc, and are recorded on an ordinal scale
policies = ox[['CountryName', 'Date', 'S1_School closing', 'S2_Workplace closing', 'S3_Cancel public events', 'S3_Cancel public events', 'S4_Close public transport', 'S5_Public information campaigns', 'S6_Restrictions on internal movement', 'S7_International travel controls']]

#the others (S8–S11) are financial indicators such as fiscal or monetary measures
financial = ox[['CountryName', 'Date', 'S8_Fiscal measures', 'S9_Monetary measures', 'S10_Emergency investment in health care', 'S11_Investment in Vaccines']]

#The tracker aggregates the policy scores (not the financial scores) into a common ‘Stringency Index’.
stringency = ox[['CountryName', 'Date', 'StringencyIndex']]

#ox_country = (ox['CountryName']).unique()


In [119]:
#Creating single country example for merging. Denmark is empty, which is an bug
ox_den = ox[ox['CountryName'] == 'Denmark']
covid_den = covid[covid['CountryName'] == 'Denmark']



#Creating single country example for merging
ox_it = ox[ox['CountryCode'] == 'ITA']
covid_it = covid[covid['CountryName'] == 'Italy']



,CountryName,CountryCode,Date,S1_School closing,S1_IsGeneral,S2_Workplace closing,S2_IsGeneral,S3_Cancel public events,S3_IsGeneral,S4_Close public transport,S4_IsGeneral,S5_Public information campaigns,S5_IsGeneral,S6_Restrictions on internal movement,S6_IsGeneral,S7_International travel controls,S8_Fiscal measures,S9_Monetary measures,S10_Emergency investment in health care,S11_Investment in Vaccines,S12_Testing framework,S13_Contact tracing,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,Unnamed: 26
7909,Italy,ITA,2020-02-23,2.0,0.0,2.0,0.0,2.0,0.0,0.0,NaN,1.0,1.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,2.0,79.0,2.0,66.67,66.67,\t


In [0]:
#reduce the variables that are not being used both in oxford and covid data
drop_ox = ['CountryName', 'CountryCode'] 
ox_it = ox_it.drop(drop_ox, axis=1)

drop_covid = ['Key', 'CountryCode'] 
covid_it = covid_it.drop(drop_covid, axis=1)

In [122]:
ox_it.head(1)

,Date,S1_School closing,S1_IsGeneral,S2_Workplace closing,S2_IsGeneral,S3_Cancel public events,S3_IsGeneral,S4_Close public transport,S4_IsGeneral,S5_Public information campaigns,S5_IsGeneral,S6_Restrictions on internal movement,S6_IsGeneral,S7_International travel controls,S8_Fiscal measures,S9_Monetary measures,S10_Emergency investment in health care,S11_Investment in Vaccines,S12_Testing framework,S13_Contact tracing,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,Unnamed: 26
7909,2020-02-23,2.0,0.0,2.0,0.0,2.0,0.0,0.0,NaN,1.0,1.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,2.0,79.0,2.0,66.67,66.67,\t


In [123]:
covid_it.head(1)

,Date,CountryName,RegionCode,RegionName,Confirmed,Deaths,Latitude,Longitude,Population
97,2019-12-31,Italy,NaN,NaN,0.0,0.0,41.87194,12.56738,60550075.0


In [145]:
dates_it = ox_it['Date'].unique()
covid_it_2 = covid_it[covid_it['Date'].isin(dates_it)]
#print(covid_it_2['Date'].unique())
covid_it_2.head(5)


,Date,CountryName,RegionCode,RegionName,Confirmed,Deaths,Latitude,Longitude,Population
6420,2020-02-23,Italy,NaN,NaN,157.0,3.0,41.871940,12.567380,60550075.0
6585,2020-02-24,Italy,NaN,NaN,229.0,7.0,41.871940,12.567380,60550075.0
6586,2020-02-24,Italy,21,Piemont,3.0,0.0,45.073274,7.680687,4377941.0
6587,2020-02-24,Italy,23,Aosta Valley,0.0,0.0,45.737503,7.320149,126933.0
6588,2020-02-24,Italy,25,Lombardy,172.0,6.0,45.466794,9.190347,10078012.0


In [143]:
print(dates_it)

['2020-02-23T00:00:00.000000000' '2020-02-24T00:00:00.000000000'
 '2020-02-25T00:00:00.000000000' '2020-02-26T00:00:00.000000000'
 '2020-02-27T00:00:00.000000000' '2020-02-28T00:00:00.000000000'
 '2020-02-29T00:00:00.000000000' '2020-03-01T00:00:00.000000000'
 '2020-03-02T00:00:00.000000000' '2020-03-03T00:00:00.000000000']


ValueError: ignored

In [0]:

#plotting statement

string = den['StringencyIndex']
cases = den['ConfirmedCases']       

string.plot.line()
#cases.plot.line()

A full description of the strengency index can be found here: 

https://www.bsg.ox.ac.uk/sites/default/files/Calculation%20and%20presentation%20of%20the%20Stringency%20Index.pdf

Additional desciption on a less detailed level: 

https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker



equals(self, other)

Test whether two objects contain the same elements.

https://pandas.pydata.org/docs/getting_started/basics.html


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

